
### Завдання 1: Виклик LLM з базовим промптом

Створіть можливість викликати LLM зі звичайним текстовим промптом.

Промпт має дозвляти отримати інформацію простою мовою на певну тему. В цьому завданні ми хочемо дізнатись про тему "Квантові обчислення".

Відповідь моделі повинна містити визначення, ключові переваги та поточні дослідження в цій галузі.

Обмежте відповідь до 200 символів і пропишіть в промпті аби відповідь була короткою (це зекономить Вам час і гроші на згенеровані токени).

В якості LLM можна скористатись як моделлю з HugginFace (рекомендую Mistral), так і ChatGPT4 або ChatGPT3. В обох випадках треба імпортувати потрібну "обгортку" (тобто клас, який дозволить ініціювати модель) з LangChain для виклику LLM за API, а також зчитати особистий токен з файла, наприклад, `creds.json`, який розміщений у Вас локально і Ви НЕ здаєте його в ДЗ і НЕ комітите в git 😏

Встановіть своє значення температури на свій розсуд (тут немає правильного чи неправильного значення) і напишіть, чому ви обрали саме таке значення для цього завдання.  

Запити можна робити як українською, так і англійською - орієнтуйтесь на те, де і чи хочете ви потім лишити цей проєкт і відповідна яка мова буде пасувати більше. В розвʼязках промпти - українською.

In [1]:
import json
import os

with open("creds.json", "r") as f:
    creds = json.load(f)
    os.environ["OPENAI_API_KEY"] = creds["OPENAI_API_KEY"]
    os.environ["SERPAPI_API_KEY"] = creds["SERPAPI_API_KEY"]

In [3]:
!pip -q install langchain langchain_openai huggingface_hub openai

In [4]:
from langchain_openai import OpenAI

llm = OpenAI(
    temperature=0.3,  # Низька температура для точності та фактичності
    max_tokens=200
)

prompt = (
    "Explain quantum computing in simple terms, including its definition, key advantages, and current research. "
    "Limit the response to 200 characters. Be concise and clear."
)

response = llm.invoke(prompt)
print(response)



Quantum computing is a type of computing that uses quantum bits (qubits) instead of traditional bits to perform calculations. It has the potential to solve complex problems much faster than classical computers. Current research focuses on developing more stable and scalable quantum systems.


### Завдання 2: Створення параметризованого промпта для генерації тексту
Тепер ми хочемо оновити попередній фукнціонал так, аби в промпт ми могли передавати тему як параметр. Для цього скористайтесь `PromptTemplate` з `langchain` і реалізуйте параметризований промпт та виклик моделі з ним.

Запустіть оновлений функціонал (промпт + модел) для пояснень про теми
- "Баєсівські методи в машинному навчанні"
- "Трансформери в машинному навчанні"
- "Explainable AI"

Виведіть результати відпрацювання моделі на екран.

In [5]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in simple terms, including its definition, key advantages, and current research. "
             "Limit the response to 200 characters. Be concise and clear."
)

topics = [
    "Bayesian methods in machine learning",
    "Transformers in machine learning",
    "Explainable AI"
]

for topic in topics:
    prompt = prompt_template.format(topic=topic)
    print(prompt.format(topic=topic))
    response = llm.invoke(prompt)
    print(f"Topic: {topic}\nResponse: {response}\n")

Explain Bayesian methods in machine learning in simple terms, including its definition, key advantages, and current research. Limit the response to 200 characters. Be concise and clear.
Topic: Bayesian methods in machine learning
Response: 

Bayesian methods in machine learning use probability to make predictions and update beliefs. They can handle uncertainty and small datasets well. Current research focuses on improving efficiency and scalability.

Explain Transformers in machine learning in simple terms, including its definition, key advantages, and current research. Limit the response to 200 characters. Be concise and clear.
Topic: Transformers in machine learning
Response: 

Transformers are a type of neural network architecture used in machine learning for natural language processing tasks. They have shown to be highly effective in tasks such as language translation and text summarization. They are known for their ability to process long sequences of data and capture complex rela



### Завдання 3: Використання агента для автоматизації процесів
Створіть агента, який допоможе автоматично шукати інформацію про останні наукові публікації в різних галузях. Наприклад, агент має знайти 5 останніх публікацій на тему штучного інтелекту.

**Кроки:**
1. Налаштуйте агента типу ReAct в LangChain для виконання автоматичних запитів.
2. Створіть промпт, який спрямовує агента шукати інформацію в інтернеті або в базах даних наукових публікацій.
3. Агент повинен видати список публікацій, кожна з яких містить назву, авторів і короткий опис.

Для взаємодії з пошуком там необхідно створити `Tool`. В лекції ми використовували `serpapi`. Можна продовжити користуватись ним, або обрати інше АРІ для пошуку (вони в тому числі є безкоштовні). Перелік різних АРІ, доступних в langchain, і орієнтир по вартості запитів можна знайти в окремому документі [тут](https://hannapylieva.notion.site/API-12994835849480a69b2adf2b8441cbb3?pvs=4).

Лишаю також нижче приклад використання одного з безкоштовних пошукових АРІ - DuckDuckGo (не потребує створення токена!)  - можливо він вам сподобається :)


In [6]:
!pip install -q langchain_community duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [7]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Obama's first name?")

'The elections of 1792 were the first ones in the United States that were contested on anything resembling a partisan basis.[20] ^ The 1796 presidential election was the first contested American presidential election and the only one in which a president and vice president were elected from opposing political parties. Barack Obama, the 44th President of the United States, broke barriers as the first African-American president and implemented significant healthcare reforms during his tenure. When the Obama family moved into the White House in 2009, Barack Obama, his wife, Michelle, and their daughters, Malia and Sasha, made up this beautiful nuclear family who were always well-dressed and well-mannered in public settings. As the first children of the country from 2009 to 2017, Malia and ... Former President Barack Obama was born in Hawaii. But nearly eight years since he left office, some social media users c Barack Obama served as the 44th president of the United States (2009-17) and w

In [8]:
!pip install -q google-search-results langchain-community langchain_experimental

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.0 MB/s eta 0:00:00


In [11]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.llms import OpenAI
import json
from langchain.utilities import SerpAPIWrapper

search = SerpAPIWrapper()
search_tool = Tool(
    name="Search",
    func=search.run,
    description="Use this tool to find the latest scientific publications on a given topic."
)

agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "5 latest scientific papers on artificial intelligence with title, authors, and summary"
result = agent.invoke(query)

print("\nResults:\n", result)



> Entering new AgentExecutor chain...
 I should use the Search tool to find the latest scientific publications on artificial intelligence
Action: Search
Action Input: "artificial intelligence"
Observation: [{'title': '1 Artificial Intelligence (AI) Stock Gen Zers Should Buy Today and Hold for Decades', 'link': 'https://www.fool.com/investing/2025/03/23/1-artificial-intelligence-ai-stock-gen-z-should-bu/', 'source': 'The Motley Fool', 'date': '2 hours ago', 'thumbnail': 'https://serpapi.com/searches/67e04847c5d2160399a6c005/images/191b2bef9224c345e1c728d187c431e4d8d37a20b2bfa63a.jpeg'}, {'title': 'AI prediction model is a major breakthrough in weather forecasting', 'link': 'https://www.earth.com/news/ai-prediction-model-is-a-major-breakthrough-in-weather-forecasting/', 'source': 'Earth.com', 'date': '1 hour ago', 'thumbnail': 'https://serpapi.com/searches/67e04847c5d2160399a6c005/images/191b2bef9224c3451394162bab3ab1a2648d01d6ed60e8fe.jpeg'}, {'title': 'AI will soon be taking your dri



### Завдання 4: Створення агента-помічника для вирішення бізнес-задач

Створіть агента, який допомагає вирішувати задачі бізнес-аналітики. Агент має допомогти користувачу створити прогноз по продажам на наступний рік враховуючи рівень інфляції і погодні умови. Агент має вміти використовувати Python і ходити в інтернет аби отримати актуальні дані.

**Кроки:**
1. Налаштуйте агента, який працюватиме з аналітичними даними, заданими текстом. Користувач пише

```
Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т. Зроби оцінку скільки ми зможемо експортувати апельсинів в 2025 враховуючи погодні умови в Бразилії і попит на апельсини в світі виходячи з економічної ситуації.
```

2. Створіть запит до агента, що містить чітке завдання – видати результат бізнес аналізу або написати, що він не може цього зробити і запит користувача (просто може бути все одним повідомлленням).

3. Запустіть агента і проаналізуйте результати. Що можна покращити?


In [15]:
from langchain_experimental.utilities import PythonREPL

search = SerpAPIWrapper()
search_tool = Tool(
    name="Search",
    func=search.run,
    description="Use this tool to get up-to-date data on the economy, inflation, and weather."
)

python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you necessarily should print it out with `print(...)`. Otherwise you won't see the result! It's very important.",
    func=python_repl.run,
)
python_tool.name = "python_interpreter"

analytics_agent = initialize_agent(
    tools=[search_tool, python_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = (
    "We export oranges from Brazil. In 2021, we exported 200 tons, in 2022 - 190 tons, in 2023 - 210 tons, "
    "in 2024, which has not yet ended - 220 tons. Make an estimate of how many oranges we will be able to export "
    "in 2025, given the weather conditions in Brazil and the demand for oranges in the world based on the economic situation."
    "Provide the reason of your decision."
    "If you can not solve this task - inform about this."
)

result = analytics_agent.invoke(query)

print("\nBusiness Analysis Result:\n", result)



> Entering new AgentExecutor chain...
 I need to find data on the economy, inflation, and weather in Brazil.
Action: Search
Action Input: "Brazil economy", "Brazil inflation", "Brazil weather"
Observation: [' entity_type: related_questions.', 'Annual inflation rate in Brazil came in at 4.83% in December 2024, little changed from 4.87% in November. Inflation softened for housing and utilities.', '4 days ago —', 'Mar 13, 2025 —', "Brazil's real GDP expanded by 2.9 percent in 2023 and is expected to grow by 2.8 percent in 2024, driven by solid consumption, sustained by a robust labor ...", 'Consumer price inflation in Brazil averaged 6.2% in the ten years to 2022, below the Latin America regional average of 8.4%. The 2022 average figure was 9.3%.', 'Feb 13, 2025 —', 'Inflation Rate MoM in Brazil averaged 5.69 percent from 1980 until 2025, reaching an all time high of 82.39 percent in March of 1990 and a record low of -0.68 ...', 'Hyperinflation in Brazil occurred between the first three

Overall the result is fine, but calculation is a bit strange.
Action: python_interpreter
Action Input: "print(200-190+210)"

It is possible to improve promt and ask take into accout the average export amount for the last few years.